In [1]:
# set up Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install a bunch of libraries
!pip install -U -q PyDrive
import pandas as pd 
import pandas_profiling
import numpy as np
import os
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
%matplotlib inline

from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from scipy.special import expit
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
import seaborn as sb
# !apt install warnings 
# warnings.filterwarnings("ignore")
!pip install  unidecode
import unidecode
import wordcloud
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
import matplotlib.animation as animation
import operator
import plotly.express as px
from collections import Counter

print("\nImported things sucessfully!")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 245kB 4.9MB/s 
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

Imported things sucessfully!


In [15]:
#show a cool report on the data
# load in the training data
# change this! 
data_file_path = "/content/drive/My Drive/Twitter Scraper Data/"
#CHANGE CSV NAME HERE 
#csv_name = "Set 1 - ALL Data 2017 - 2020.csv"
csv_name = "Set 2 - Shutdown Data 12-8-2017 to 2-8-2020.csv"

final_file_path = data_file_path + csv_name
df=pd.read_csv(final_file_path, delimiter=',')
df.head()


#pandas_profiling.ProfileReport(df)

#df = pd.read_csv('Datasets/BL-Flickr-Images-Book.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:55:50,UTC,2.545345e+09,3duard0n,Adventures on Wheels,"Joshua Tree, CA",Exotic rock formations at Joshua Tree National...,[],['https://www.instagram.com/p/BP8q02sD2pb/'],[],0,0,0,[],[],https://twitter.com/3duard0N/status/8265797774...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '2545344619', 'username': '3duard...",NaN,NaN,NaN,NaN
1,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:48:36,UTC,2.034075e+07,julia_laflin,Julia Laflin,NaN,"Hidden valleys, cattle rustling, old gold mini...",[],[],['https://pbs.twimg.com/media/C3iYPnvUoAAjuUy....,0,0,2,"['#joshuatree', '#california', '#amwriting', '...",[],https://twitter.com/Julia_Laflin/status/826577...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '20340746', 'username': 'Julia_La...",NaN,NaN,NaN,NaN
2,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,UTC,6.048998e+08,shutterangle180,Kurt Hall,NaN,Please help fund this important film! http://...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf9hWEAAD1VM....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",[],https://twitter.com/shutterangle180/status/826...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '604899840', 'username': 'shutter...",NaN,NaN,NaN,NaN
3,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,UTC,5.338192e+08,tourshollywood,Designated Place,NaN,Please help fund this important film! http://...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf3jWcAAMehy....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",[],https://twitter.com/ToursHollywood/status/8265...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '533819191', 'username': 'ToursHo...",NaN,NaN,NaN,NaN
4,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:40:12,UTC,1.528575e+09,bellandromeda,spider-bellaðŸ•¸,NaN,My lil bebe is healing nicely ðŸ˜ I can't wai...,[],[],['https://pbs.twimg.com/media/C3iWUwwXAAM0KsY....,0,0,2,[],[],https://twitter.com/bellandromeda/status/82657...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1528574792', 'username': 'bellan...",NaN,NaN,NaN,NaN


In [16]:
# remove columns
# to add more, include column name below 
df=df.drop(columns=['timezone','name','cashtags','retweet','video','near', 'geo','source','user_rt_id','user_rt','retweet_id','reply_to','retweet_date','translate','trans_src','trans_dest'])

df.head()

,id,conversation_id,created_at,date,time,user_id,username,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,link,quote_url
0,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:55:50,2.545345e+09,3duard0n,"Joshua Tree, CA",Exotic rock formations at Joshua Tree National...,[],['https://www.instagram.com/p/BP8q02sD2pb/'],[],0,0,0,[],https://twitter.com/3duard0N/status/8265797774...,NaN
1,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:48:36,2.034075e+07,julia_laflin,NaN,"Hidden valleys, cattle rustling, old gold mini...",[],[],['https://pbs.twimg.com/media/C3iYPnvUoAAjuUy....,0,0,2,"['#joshuatree', '#california', '#amwriting', '...",https://twitter.com/Julia_Laflin/status/826577...,NaN
2,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,6.048998e+08,shutterangle180,NaN,Please help fund this important film! http://...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf9hWEAAD1VM....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",https://twitter.com/shutterangle180/status/826...,NaN
3,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,5.338192e+08,tourshollywood,NaN,Please help fund this important film! http://...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf3jWcAAMehy....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",https://twitter.com/ToursHollywood/status/8265...,NaN
4,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:40:12,1.528575e+09,bellandromeda,NaN,My lil bebe is healing nicely ðŸ˜ I can't wai...,[],[],['https://pbs.twimg.com/media/C3iWUwwXAAM0KsY....,0,0,2,[],https://twitter.com/bellandromeda/status/82657...,NaN


In [0]:
# making all tweets lowercase 
# take this out! 
# df['tweet'] = df['tweet'].str.lower()
#df.head()


In [0]:
df['tweet']=df['tweet'].fillna("N/A")

In [18]:
# removing random characters we no longer need 
# to add more, include characters below
df['tweet']=df['tweet'].replace(to_replace=['€','˜','~','™'], value=' ', regex=True)

#Removing all the greek characters using unidecode library
df['tweet'] = df['tweet'].apply(lambda x : ' '.join([unidecode.unidecode(word) for word in x.split()]))

#Removing the word 'hmm' and it's variants
df['tweet'] = df['tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word == 'h(m)+' ]))

#Removing slang words - Replace "luv" with "love" etc
d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
               'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
               'cud':'could','fud':'food'} ## Need a huge dictionary
words1 = "I luv myself"
words = words1.split()
reformed = [d[word] if word in d else word for word in words]
reformed = " ".join(reformed)
print("The phrase '" + words1 +"' will be transformed to '"+reformed+"', along with some other slang word combinations")

df['tweet'] = df['tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))


df.head()

The phrase 'I luv myself' will be transformed to 'I love myself', along with some other slang word combinations


,id,conversation_id,created_at,date,time,user_id,username,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,link,quote_url
0,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:55:50,2.545345e+09,3duard0n,"Joshua Tree, CA",Exotic rock formations at Joshua Tree National...,[],['https://www.instagram.com/p/BP8q02sD2pb/'],[],0,0,0,[],https://twitter.com/3duard0N/status/8265797774...,NaN
1,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:48:36,2.034075e+07,julia_laflin,NaN,"Hidden valleys, cattle rustling, old gold mini...",[],[],['https://pbs.twimg.com/media/C3iYPnvUoAAjuUy....,0,0,2,"['#joshuatree', '#california', '#amwriting', '...",https://twitter.com/Julia_Laflin/status/826577...,NaN
2,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,6.048998e+08,shutterangle180,NaN,Please help fund this important film! http://g...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf9hWEAAD1VM....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",https://twitter.com/shutterangle180/status/826...,NaN
3,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,5.338192e+08,tourshollywood,NaN,Please help fund this important film! http://g...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf3jWcAAMehy....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",https://twitter.com/ToursHollywood/status/8265...,NaN
4,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:40:12,1.528575e+09,bellandromeda,NaN,My lil bebe is healing nicely dY I can't wait ...,[],[],['https://pbs.twimg.com/media/C3iWUwwXAAM0KsY....,0,0,2,[],https://twitter.com/bellandromeda/status/82657...,NaN


In [19]:
letters_only_sub_string = "[^a-zA-Z?!#:.,'0-9$-<>&;]"  #String that determines what characters we keep.  Currently getting rid of all numbers and non a-z characters except what you add in
df['tweet'] = df['tweet'].apply(lambda x : ' '.join([re.sub('http\S+', '', x)]))
df['tweet'] = df['tweet'].apply(lambda x : ' '.join([re.sub('pic.twitter.com\S+', '', x)]))
df['tweet'] = df['tweet'].apply(lambda x : ' '.join([re.sub(letters_only_sub_string,' ', x)]))
df.head()

,id,conversation_id,created_at,date,time,user_id,username,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,link,quote_url
0,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:55:50,2.545345e+09,3duard0n,"Joshua Tree, CA",Exotic rock formations at Joshua Tree National...,[],['https://www.instagram.com/p/BP8q02sD2pb/'],[],0,0,0,[],https://twitter.com/3duard0N/status/8265797774...,NaN
1,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:48:36,2.034075e+07,julia_laflin,NaN,"Hidden valleys, cattle rustling, old gold mini...",[],[],['https://pbs.twimg.com/media/C3iYPnvUoAAjuUy....,0,0,2,"['#joshuatree', '#california', '#amwriting', '...",https://twitter.com/Julia_Laflin/status/826577...,NaN
2,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,6.048998e+08,shutterangle180,NaN,Please help fund this important film! #Suppor...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf9hWEAAD1VM....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",https://twitter.com/shutterangle180/status/826...,NaN
3,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:45:14,5.338192e+08,tourshollywood,NaN,Please help fund this important film! #Suppor...,[],['http://gofundme.com/designatedplace'],['https://pbs.twimg.com/media/C3iXf3jWcAAMehy....,0,0,0,"['#supportindiefilm', '#indiefilm', '#document...",https://twitter.com/ToursHollywood/status/8265...,NaN
4,8.270000e+17,8.270000e+17,1.490000e+12,1/31/2017,23:40:12,1.528575e+09,bellandromeda,NaN,My lil bebe is healing nicely dY I can't wait ...,[],[],['https://pbs.twimg.com/media/C3iWUwwXAAM0KsY....,0,0,2,[],https://twitter.com/bellandromeda/status/82657...,NaN


In [20]:
#save the cleaned csv file 
clean_csv_name = "clean_" + csv_name 
clean_final_file_path = data_file_path + clean_csv_name
df.to_csv(clean_final_file_path, encoding='utf-8', index=False)
clean_final_file_path

'/content/drive/My Drive/Twitter Scraper Data/clean_Set 2 - Shutdown Data 12-8-2017 to 2-8-2020.csv'